# Netflix Movies, IMDB and Rotten Tomatoes Rating, Good Books ETL
A lot of people check IMDB or Rotten Tomatoes score for a movie before watching it on Netflix. Although Netflix has a good recommender system, it still can be hard to use for people to determine how good a movie is. The exercise collected Netflix movies data along with IMDB and Rotten Tomatoes rating data, cleaned them and loaded them into a SQL database, so further analysis can be performed to identify top rated movies on Netflix based on IMDB and Rotten Tomatoes ratings. The database also included a 10k good books list to identify which movies were produced from popular books.


In [ ]:
import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")
from sqlalchemy import create_engine

# Data Extract

### Import CSV files from resource folder
Data Source:
1.	https://www.kaggle.com/shivamb/netflix-shows?select=netflix_titles.csv
2.	https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset
3.	https://www.kaggle.com/stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset?select=rotten_tomatoes_movies.csv
4.	https://www.kaggle.com/zygmunt/goodbooks-10k?select=books.csv

In [ ]:
netflix_all = pd.read_csv('Resources/netflix_titles.csv')
netflix_all.head()

In [ ]:
rotten_tomatoes_movies = pd.read_csv('Resources/rotten_tomatoes_movies.csv')
rotten_tomatoes_movies.head(2)

In [ ]:
csv_file = "Resources/IMDb ratings.csv"
ratings_data_df = pd.read_csv(csv_file)
ratings_data_df.head()

In [ ]:
movies_file ="Resources/IMDb movies.csv"
movies_data_df = pd.read_csv(movies_file)
movies_data_df.head(2)

In [ ]:
books_file ="Resources/books.csv"
books_data_df = pd.read_csv(books_file)
books_data_df.head()

# Data Transformation 

### Netflix Data File Transformation: 
1. Checked the data types - Changed "date_added" to datetime format
2. Filtered out movies from all show types by loc function
3. Renamed, organized and split the data into two tables:
    - Netflix Movie Basic Columns: netflix_show_id,title,country,rating,duration,description
    - Netflix Movie Date Columns: netflix_show_id,date_added,release_year
4. Checked and handled missing values for each new table:
    - Replaced 230 missing "country" values to "other"
    - Replaced 5 missing "rating" values to "NR"

In [ ]:
netflix_all.dtypes

In [ ]:
# Change value type tpo datetime
netflix_all['date_added'] =pd.to_datetime(netflix_all.date_added)

#Check overall missing values
netflix_all.isnull().sum()

In [ ]:
# Filter out Movies on Netflix
netflix_movie = netflix_all.loc[netflix_all.type=='Movie']
netflix_movie.shape

In [ ]:
# Rename show id column for database consistency
netflix_movie = netflix_movie.rename(columns={'show_id':'netflix_show_id'})

In [ ]:
# Select and split dataframe into two tables by content
netflix_movie_basic = netflix_movie [[ 'netflix_show_id','title','country','rating','duration','description']]
netflix_movie_basic.isnull().sum()

In [ ]:
netflix_movie_basic.rating.value_counts()

In [ ]:
# Change the countries from Na to "other"; 5 missing ratings to "NR"
netflix_movie_basic['country'] =netflix_movie_basic['country'].fillna('other')
netflix_movie_basic['rating'] =netflix_movie_basic['rating'].fillna('NR')

In [ ]:
netflix_movie_basic.isnull().sum()

In [ ]:
netflix_movie_basic.set_index("netflix_show_id", inplace=True)
netflix_movie_basic.head()

In [ ]:
netflix_movie_date = netflix_movie [[ 'netflix_show_id','date_added','release_year']]
netflix_movie_date.isnull().sum()

In [ ]:
# set id as index for PK
netflix_movie_date.set_index("netflix_show_id", inplace=True)
netflix_movie_date.head()

### Rotten Tomatoes Movies Data File Transformation:

1. Checked the data types
2. Selected columns and kept "title", "rating" and "counts" only
3. Renamed "id" and "title" columns for label consistency within the database
4. Checked missing values and dropped 200+ NA rows related to "voting" columns only

In [ ]:
# Rename show id and title columns for database consistency
rotten_tomatoes_movies = rotten_tomatoes_movies.rename(columns={'rotten_tomatoes_link':'rotten_tomatoes_id','movie_title':'title'})
rotten_tomatoes_movies.shape

In [ ]:
rotten_tomatoes_movies.dtypes

In [ ]:
# Check overall missing values
rotten_tomatoes_movies.isnull().sum()

In [ ]:
# Drop null values for selected rating columns only. Fill null values for "audience status" as "other"
rotten_tomatoes_movies_clean = rotten_tomatoes_movies.dropna(subset = ['tomatometer_rating', 'audience_count'])
rotten_tomatoes_movies_clean['audience_status'] =rotten_tomatoes_movies_clean['audience_status'].fillna('other')

In [ ]:
# Select rating related columns as new dataframe and set id as index for PK
rotten_tomatoes_movies_rating = rotten_tomatoes_movies_clean[['rotten_tomatoes_id','title','tomatometer_status','tomatometer_rating','tomatometer_count','audience_status','audience_rating','audience_count']]
rotten_tomatoes_movies_rating.set_index("rotten_tomatoes_id", inplace=True)
rotten_tomatoes_movies_rating.head(2)

### IMDB Movie Data Transformation:
1. Created a new dataFrame by selecting and renaming "id" and "title" columns for database consistency
3. Set ID as index for PK

In [ ]:
#Checking columns names
movies_data_df.columns

In [ ]:
movies_data_df.isnull().sum()

In [ ]:
movies_df = movies_data_df[["imdb_title_id", "title"]].copy()
movies_df.head()

In [ ]:
movies_df.info()

In [ ]:
movies_df['title'].value_counts()
movies_df.drop_duplicates("title", inplace=True)

In [ ]:
movies_df['title'].value_counts()

In [ ]:
# movies_df["movie_title"]=movies_df["movie_title"].fillna("unknown")
# unknown = movies_df.loc[movies_df["movie_title"]=="unknown"]
# unknown.count()
movies_df = movies_df.rename(columns= {"imdb_title_id":"id", "title":"movie_title"})
movies_df.set_index("id", inplace = True)
movies_df.head()

### IMDB Data Rating File Transformation:
1. Removed irrelevant columns and reorganized the votes by gender regardless of age
2. The "mean_vote" and "median_vote" columns were also removed due to redundancy
3. The "total_votes" and "weighted_average_vote" columns were re-ordered for better clarity 

In [ ]:
# Keep rating related columns only
imdb_ratings_data_df = ratings_data_df[['imdb_title_id', 'total_votes', 'weighted_average_vote', 'males_allages_avg_vote', 'males_allages_votes', 'females_allages_avg_vote', 'females_allages_votes']].copy()

In [ ]:
# Set id as index as PK
imdb_ratings_data_df.set_index ("imdb_title_id", inplace = True)

In [ ]:
# FOr this excercise we focus on total votes and weighted average votes
imdb_ratings_data_df.isnull().sum()

### Books File Transformation

1. Created a book dataframe with selecting "title", "authors" and "publication year" columns
2. Dropped duplicate books by "title"
3. Checked and handled 20 missing values in "original_publication_year"
4. Renamed columns for clarification
5. Set ID as index for PK

In [ ]:
books_data_df.columns

In [ ]:
books_data_df.info()

In [ ]:
books_data_df.isnull().sum()

In [ ]:
books_df = books_data_df[["book_id", "authors", "title", "original_publication_year"]].copy()
books_df.head()

In [ ]:
books_df["original_publication_year"]=books_df["original_publication_year"].fillna(0)
books_df.isnull().sum()

In [ ]:
books_df.title.value_counts()

In [ ]:
books_df.drop_duplicates(["title"], inplace=True)
books_df.title.value_counts()

In [ ]:
books_df = books_df.rename(columns= {"book_id":"ID", "authors":"Authors", "title":"Book_Title", "original_publication_year":"Published_Year"})
books_df.set_index("ID", inplace = True)
books_df.head()

In [ ]:
books_df["Published_Year"]=books_df["Published_Year"].astype(int)
books_df.dtypes

# Load Data to SQL

Create database connection to SQL and pushed all tranformed tables into SQL database "Netflix_IMDB"

In [ ]:
#Create database connection 
connection_string = "postgres:bootcamp@localhost:5432/Netflix_IMDB"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Load all tables into SQL database
netflix_movie_date.to_sql(name='netflix_movie_date', con=engine, if_exists='replace', index=True)

In [ ]:
netflix_movie_basic.to_sql(name='netflix_movie_basic', con=engine, if_exists='replace', index=True)

In [ ]:
rotten_tomatoes_movies_rating.to_sql(name='rotten_tomatoes_movies_rating', con=engine, if_exists='replace', index=True)

In [ ]:
imdb_ratings_data_df.to_sql(name='imdb_ratings', con=engine, if_exists='replace', index=True)

In [ ]:
movies_df.to_sql(name="imdb_movies", con=engine, if_exists="replace", index=True)

In [ ]:
books_df.to_sql(name="books", con=engine, if_exists= "replace", index=True)

# Confirm Data Loaded

In [ ]:
pd.read_sql_query("select * from netflix_movie_date", con=engine).head()

In [ ]:
pd.read_sql_query("select * from netflix_movie_basic", con=engine).head()

In [ ]:
pd.read_sql_query("select * from rotten_tomatoes_movies_rating", con=engine).head()

In [ ]:
pd.read_sql_query("select * from imdb_ratings", con=engine).head()

In [ ]:
pd.read_sql_query("select * from imdb_movies", con=engine).head()

In [ ]:
pd.read_sql_query("select * from imdb_movies", con=engine).head()